In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import sklearn.ensemble as skensemble
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [3]:
titanic = pd.DataFrame.from_csv("train_processed.csv")

feature_names = ["Pclass","Age","SibSp","Parch","Fare","IsMale"]
Xtrain = titanic[feature_names]
ytrain = titanic["Survived"]

In [4]:
loss = ['deviance', 'exponential']
learning_rate = np.logspace(-4,3)
n_estimate_dist = sp_randint(100,1000)
max_depth_dist = sp_randint(1,5)
param_dist = dict(loss=loss,
                learning_rate=learning_rate,
                n_estimators=n_estimate_dist,
                max_depth=max_depth_dist)

In [5]:
gbdt = skensemble.GradientBoostingClassifier()
n_iter_search = 200
searchcv = RandomizedSearchCV(estimator=gbdt, param_distributions=param_dist,n_iter=n_iter_search)
searchcv.fit(Xtrain,ytrain)    

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
          fit_params={}, iid=True, n_iter=200, n_jobs=1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000000186EAFD0>, 'loss': ['deviance', 'exponential'], 'learning_rate': array([  1.00000e-04,   1.38950e-04,   1.93070e-04,   2.68270e-04,
         3.72759e-04,   5.17947e-04,   7.19686e-04,   1.00000e-03,...0000e+03]), 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000000186EAF98>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring=None, verbose=0)

In [6]:
searchcv.best_score_                                  

0.82603815937149272

In [7]:
searchcv.best_estimator_

GradientBoostingClassifier(init=None, learning_rate=0.01, loss='exponential',
              max_depth=4, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=587,
              random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [8]:
searchcv.best_params_

{'learning_rate': 0.01,
 'loss': 'exponential',
 'max_depth': 4,
 'n_estimators': 587}

In [9]:
import pickle
f = open('gbdt3.pkl', 'wb')
pickle.dump(searchcv.best_estimator_,f)
f.close()

In [10]:
titanic_test = pd.DataFrame.from_csv("test_processed.csv")
Xtest = titanic_test[feature_names]

predictions = searchcv.predict(Xtest)
submission = pd.DataFrame({
        "PassengerId": titanic_test["PassengerId"],
        "Survived": predictions
    })
submission.to_csv("submit_gbdt3.csv", index=False)